# Comparaison des Trendlines

Ce notebook compare les trendlines générées par le scraping des marketplaces avec celles calculées à partir des données SoH de Bib.

## Import data

In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from core.sql_utils import *
from core.gsheet_utils import *
from results.trendline.trendline_utils import *
from core.numpy_utils import numpy_safe_eval


In [ ]:
df_scrapping = load_excel_data( "Courbes de tendance", "Courbes OS", )
df_scrapping = pd.DataFrame(columns=df_scrapping[0,:8], data=df_scrapping[1:,:8])
df_scrapping['origin'] = 'scrapping'
df_scrapping.rename(columns={'OEM' : 'make_name',
                             'Modèle': 'model_name',
                             'Type': 'type',
                             'Odomètre (km)': 'odometer',
                             'SoH': 'soh', }, inplace=True)

df_scrapping['model_name'] = df_scrapping['model_name'].apply(lambda x: x.lower())
df_scrapping['make_name'] = df_scrapping['make_name'].apply(lambda x: x.lower())
df_scrapping['odometer'] = df_scrapping['odometer'].apply(lambda x: float(x.replace(',', '')))
df_scrapping['soh'] = df_scrapping['soh'].apply(lambda x: float(x.replace('%', ''))/100)
df_scrapping = df_scrapping.replace('unknown', np.nan)
df_scrapping.dropna(subset=['model_name', 'make_name', 'odometer', 'soh'], inplace=True)
df_scrapping.drop_duplicates(inplace=True)

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT vm.model_name, vm.id, vm.type, m.make_name, b.capacity FROM vehicle_model vm
                                  left join make m on m.id=vm.make_id
                                  left join battery b on b.id=vm.battery_id;"""), con)
dbeaver_df = dbeaver_df.groupby(['model_name', 'type'], as_index=False).first()
with engine.connect() as connection:
    prod_df = pd.read_sql(text("""SELECT v.vin, vm.model_name, vm.type, vd.odometer, vd.soh, vm.version, b.capacity, m.make_name from vehicle_data vd
            left join vehicle v
            on vd.vehicle_id=v.id
            left join vehicle_model vm
            on v.vehicle_model_id=vm.id
            left join battery b on b.id=vm.battery_id
            left join make m on m.id=vm.make_id;"""), con)
    prod_df['origin'] = 'bib'
    prod_df.dropna(subset=['model_name', 'make_name', 'odometer', 'soh'], inplace=True)


In [ ]:
df_scrapping = df_scrapping.merge(dbeaver_df, on=['model_name', 'make_name', 'type'], how='left')

In [ ]:
df = pd.concat([df_scrapping[['model_name', 'make_name', 'type', 'soh', 'origin', 'capacity', 'odometer']], prod_df])
df

In [ ]:
res = df.groupby('model_name')[['origin']].value_counts()
res = pd.DataFrame(res).reset_index()

In [ ]:
res['model_name'].value_counts()[res['model_name'].value_counts()>1].index

In [ ]:
rep = {}
for model in res['model_name'].value_counts()[res['model_name'].value_counts()>1].index:
    mask = df['model_name'] == model
    df_temp = df[mask]
    rep[model] = df_temp['origin'].value_counts()


## Trendline comparison

### Compute trendline

In [ ]:
def compare_trendlines_for_model(df, model_name, display_title=None):
    """
    Compare les trendlines pour un modèle spécifique entre 3 sources:
    - Données BIB uniquement
    - Données scraping uniquement
    - Données combinées (BIB + scraping)

    Args:
        df: DataFrame contenant toutes les données
        model_name: Nom du modèle à analyser
        display_title: Titre optionnel pour le graphique (utilise model_name par défaut)

    Returns:
        fig: Figure Plotly avec les 3 trendlines
    """
    # Filtrer les données pour le modèle
    mask = df['model_name'] == model_name
    df_model = df[mask]

    # Afficher la répartition des données
    print(f"\n=== {display_title or model_name.upper()} ===")
    print(df_model['origin'].value_counts())

    # Calculer les trendlines pour chaque source
    try:
        trendline_bib = compute_main_trendline(
            df_model[df_model['origin'] == 'bib']['odometer'].sort_values(),
            df_model[df_model['origin'] == 'bib']['soh'].sort_values()
        )
    except Exception as e:
        print(f"Erreur calcul trendline BIB: {e}")
        trendline_bib = None

    try:
        trendline_scrapping = compute_main_trendline(
            df_model[df_model['origin'] == 'scrapping']['odometer'].sort_values(),
            df_model[df_model['origin'] == 'scrapping']['soh'].sort_values()
        )
    except Exception as e:
        print(f"Erreur calcul trendline scraping: {e}")
        trendline_scrapping = None

    try:
        trendline_both = compute_main_trendline(
            df_model['odometer'].sort_values(),
            df_model['soh'].sort_values()
        )
    except Exception as e:
        print(f"Erreur calcul trendline combinée: {e}")
        trendline_both = None

    # Créer le graphique
    fig = go.Figure()

    odometer_range = pd.Series([i for i in range(1, 200000, 1000)])

    if trendline_bib:
        fig.add_trace(go.Scatter(
            x=odometer_range,
            y=numpy_safe_eval(expression=trendline_bib[3]['trendline'], x=odometer_range),
            mode='lines',
            name='BIB',
            line=dict(color='blue', width=2)
        ))

    if trendline_scrapping:
        fig.add_trace(go.Scatter(
            x=odometer_range,
            y=numpy_safe_eval(
                expression=trendline_scrapping[3]["trendline"], x=odometer_range
            ),
            mode="lines",
            name="Scraping",
            line=dict(color="red", width=2),
        ))

    if trendline_both:
        fig.add_trace(go.Scatter(
            x=odometer_range,
            y=numpy_safe_eval(
                expression=trendline_both[3]["trendline"], x=odometer_range
            ),
            mode="lines",
            name="Combiné",
            line=dict(color="green", width=2, dash="dash"),
        ))

    fig.update_layout(
        title=f"Comparaison Trendlines - {display_title or model_name.title()}",
        xaxis_title='Kilométrage (km)',
        yaxis_title='SoH (%)',
        hovermode='x unified',
        template='plotly_white'
    )

    return fig


## Comparaisons par Modèle


###  Renault Zoe

In [ ]:
fig = compare_trendlines_for_model(df, 'zoe', 'Renault Zoe')
fig.show()

### Tesla Model Y


In [ ]:
fig = compare_trendlines_for_model(df, 'model y', 'Tesla Model Y')
fig.show()


### Mercedes Sprinter


In [ ]:
fig = compare_trendlines_for_model(df, 'sprinter', 'Mercedes Sprinter')
fig.show()


### 4.4 Tesla Model 3


In [ ]:
fig = compare_trendlines_for_model(df, 'model 3', 'Tesla Model 3')
fig.show()


## 5. Conclusion

Ce notebook permet de comparer les trendlines de dégradation du SoH pour différents modèles de véhicules électriques en utilisant trois sources de données :
- **BIB** : Données collectées directement depuis les véhicules connectés
- **Scraping** : Données extraites des annonces de vente sur les marketplaces
- **Combiné** : Fusion des deux sources pour avoir une vue d'ensemble

`Résultats par modèles`

- Pour **Tesla**, le nombre de SoH scrappés n’est pas représentatif par rapport aux SoH calculés par Bib, mais on observe que la *trendline de Bib est 3 points supérieure* pour le **Model Y** et *5 points supérieure* pour **le Model 3** par rapport a celle du scrapping.

- Pour les **Renault Zoé**, on constate que la trendline de *Bib est 1 point inférieure à celle issue du scrapping*.

- Pour les **Mercedes Sprinter**, la trendline de Bib n’a pas de sens (résultats incohérents).
